In [2]:
import requests
import pandas as pd
from plyer import notification


def buscar_json_do_endpoint(url_endpoint):
    # Busca dados JSON do URL do endpoint especificado.
    try:
        resposta = requests.get(url_endpoint)
        resposta.raise_for_status()
        return resposta.json()
    except requests.RequestException as erro:
        print(f"Falha na requisição: {erro}")
        return None


def converter_para_dataframe(dados):
    # Converte os dados fornecidos em um DataFrame do pandas.
    return pd.DataFrame(dados)


def expandir_colunas_regiao(dataframe):
    # Expande a coluna 'regiao'
    if 'regiao' in dataframe.columns:
        regiao_expandida = dataframe['regiao'].apply(pd.Series)
        regiao_expandida.columns = ['regiao_id', 'regiao_sigla', 'regiao_nome']
        dataframe = pd.concat([dataframe, regiao_expandida], axis=1).drop(columns=['regiao'])
    return dataframe


def buscar_dados_municipio(sigla_estado):
    # https://brasilapi.com.br/api/ibge/municipios/v1/{sigla_estado}?providers=dados-abertos-br,gov,wikipedia - Dados desse endpoint
    url_api = f"https://brasilapi.com.br/api/ibge/municipios/v1/{sigla_estado}?providers=dados-abertos-br,gov,wikipedia"
    dados_json = buscar_json_do_endpoint(url_api)
    if dados_json:
        dataframe = converter_para_dataframe(dados_json)
        dataframe['sigla'] = sigla_estado
        return dataframe
    return pd.DataFrame()


def criar_df_final(url_dados_estado):
    dados_estado = buscar_json_do_endpoint(url_dados_estado)
    if dados_estado is None:
        return None

    dataframe_estado = converter_para_dataframe(dados_estado)
    dataframe_estado = expandir_colunas_regiao(dataframe_estado)

    dataframes_municipios = []
    for sigla_estado in dataframe_estado['sigla']:
        dataframe_municipio = buscar_dados_municipio(sigla_estado)
        if not dataframe_municipio.empty:
            dataframes_municipios.append(dataframe_municipio)

    if not dataframes_municipios:
        print("Nenhum dado de município foi buscado.")
        return None

    dataframe_municipios = pd.concat(dataframes_municipios, ignore_index=True)

    dataframe_final = pd.merge(dataframe_municipios, dataframe_estado, on='sigla', how='inner')
    dataframe_final = dataframe_final.rename(columns={
        "nome_x": "municipio_nome",
        "nome_y": "estado_nome",
        "sigla": "estado_sigla",
        "id": "estado_id",
    })

    dataframe_final = dataframe_final.sort_values(by=['regiao_id', 'estado_nome'], ascending=True)
    return dataframe_final


def buscar_dados_bancos():
    # https://brasilapi.com.br/api/banks/v1 - Dados desse endpoint
    url_api_bancos = "https://brasilapi.com.br/api/banks/v1"
    dados_json_bancos = buscar_json_do_endpoint(url_api_bancos)
    if dados_json_bancos:
        return converter_para_dataframe(dados_json_bancos)
    return pd.DataFrame()


def enviar_alerta(mensagem):
    #Envia uma notificação de alerta.
    notification.notify(
        title='Alerta',
        message=mensagem,
        timeout=10
    )


url_dados_estado = "https://brasilapi.com.br/api/ibge/uf/v1"
dataframe_estado_final = criar_df_final(url_dados_estado)

if dataframe_estado_final is not None and not dataframe_estado_final.empty:
    display(dataframe_estado_final)
    enviar_alerta("Dados dos estados e municípios processados com sucesso.")
else:
    print("Falha ao recuperar ou processar dados dos estados e municípios.")
    enviar_alerta("Falha ao processar dados dos estados e municípios.")

# Busca e exibe dados dos bancos
dataframe_bancos = buscar_dados_bancos()

if dataframe_bancos is not None and not dataframe_bancos.empty:
    display(dataframe_bancos)
    enviar_alerta("Dados dos bancos foram coletados com sucesso.")
else:
    print("Falha ao recuperar dados dos bancos.")
    enviar_alerta("Falha ao coletar dados dos bancos.")


,municipio_nome,codigo_ibge,estado_sigla,estado_id,estado_nome,regiao_id,regiao_sigla,regiao_nome
52,ACRELANDIA,1200013,AC,12,Acre,1,N,Norte
53,ASSIS BRASIL,1200054,AC,12,Acre,1,N,Norte
54,BRASILEIA,1200104,AC,12,Acre,1,N,Norte
55,BUJARI,1200138,AC,12,Acre,1,N,Norte
56,CAPIXABA,1200179,AC,12,Acre,1,N,Norte
...,...,...,...,...,...,...,...,...
5177,TACURU,5007950,MS,50,Mato Grosso do Sul,5,CO,Centro-Oeste
5178,TAQUARUSSU,5007976,MS,50,Mato Grosso do Sul,5,CO,Centro-Oeste
5179,TERENOS,5008008,MS,50,Mato Grosso do Sul,5,CO,Centro-Oeste
5180,TRÊS LAGOAS,5008305,MS,50,Mato Grosso do Sul,5,CO,Centro-Oeste


,ispb,name,code,fullName
0,00000000,BCO DO BRASIL S.A.,1.0,Banco do Brasil S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70.0,BRB - BANCO DE BRASILIA S.A.
2,00038121,Selic,NaN,Banco Central do Brasil - Selic
3,00038166,Bacen,NaN,Banco Central do Brasil
4,00122327,SANTINVEST S.A. - CFI,539.0,"SANTINVEST S.A. - CREDITO, FINANCIAMENTO E INV..."
...,...,...,...,...
354,92875780,WARREN CVMC LTDA,371.0,WARREN CORRETORA DE VALORES MOBILIÁRIOS E CÂMB...
355,92894922,BANCO ORIGINAL,212.0,Banco Original S.A.
356,94968518,EFX CC LTDA.,289.0,EFX CORRETORA DE CÂMBIO LTDA.
357,,NaN,NaN,NaN
